In [55]:
import numpy as np
from scipy.special import expit as sigmoid
from typing import List, Union, Callable
import sys

In [4]:
X = np.identity(8)
y = X

In [124]:
def init_network(input_size, hidden_size, output_size):
    input = np.zeros((input_size, 2))
    hidden = np.zeros((hidden_size, 2))
    output = np.zeros((output_size, 2))
    network = list((input, hidden, output))
    return network
network = init_network(8,3,8)
network

[array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])]

In [67]:
def sigmoid_derivative(z: float) -> float:
    return sigmoid(z)*(1-sigmoid(z))

def quadratic_loss(predictions: np.ndarray, actuals: np.ndarray) -> np.ndarray:
    norms = np.apply_along_axis(np.linalg.norm, 0, predictions-actuals)
    return 0.5*np.apply_along_axis(np.power, 0, norms, {'x2': 2})

def cost_function(loss_function: Callable, predictions: np.ndarray, actuals: np.ndarray, weigths: np.ndarray, decay_parameter: float) -> float:
    avg_loss: float = np.mean(loss_function(predictions, actuals))
    regularization: float = 0.5*decay_parameter*sum(np.power(weights, 2))
    return avg_loss+regularization

class Layer:

    # declaration of instance variables
    weigths: np.ndarray
    has_bias: bool

    def __init__(self, num_nodes: int, num_nodes_n1: int, include_bias: bool = True, epsilon: float = 0.01) -> None:
        self.weights = np.random.normal(loc=0, scale=np.power(epsilon,2), size=(num_nodes_n1, np.add(num_nodes, include_bias)))
        self.has_bias = include_bias
    
    def print_weights(self) -> None:
        print(self.weights)


class Network:

    # declaration of instance variables
    layers: List[Layer]

    def __init__(self, num_nodes: List[int], include_biases: List[bool]) -> None:
        # num_nodes is a list of number of nodes for all layers not counting the bias node
        # TO-DO: code won't work if include_biases != [True, True, False], see prop_forward
        assert (include_biases == [True, True, False]), 'error when initializing Network class: include_bias parameter not available'

        self.layers = [Layer(num_nodes[i], num_nodes[i+1], include_biases[i]) for i in range(len(num_nodes)-1)]
        self.layers.append(Layer(num_nodes[-1], 0, include_biases[-1]))

    def get_weights(self, form: str = 'vector') -> Union[List[np.ndarray], np.ndarray]:
        assert (form in ['vector', 'list']), 'Error in get_weights function: form parameter ill-defined'

        if form == 'vector':
            # returns one np.ndarray with all weights of all layers
            weigth_vector = []
            for layer in self.layers:
                weigth_vector.append(layer.weights)
            return np.asarray(weigth_vector)
        elif form == 'list':
            # returns a list, where list[i] stores the weights between layer i-1 and layer i
            list_of_weights: List[np.ndarray] = []
            for layer in self.layers:
                list_of_weights.append(layer.weights)
            return list_of_weights

    def print_weights(self) -> None:
        print('Printing weights of network:')
        for index, layer in enumerate(self.layers):
            print(f'Layer {index+1}')
            layer.print_weights()

    def prop_forward(self, features: np.ndarray) -> List[np.ndarray]:
        # returns a list, where list[i] stores the activations for neurons in layer i+1
        # the activation of a bias node (should the layer have one) is given by the first value in the array and is always =1
        # TO-DO: right now it is hard coded that input layer & hidden layer have a bias node, but output layer has not
        z_2 = np.matmul(self.layers[0].weights, np.append(1, features))
        a_2 = np.apply_along_axis(sigmoid, 0, z_2)
        z_3 = np.matmul(self.layers[1].weights, np.append(1, a_2))
        a_3 = np.apply_along_axis(sigmoid, 0, z_3)
        return [np.append(1, features), np.append(1, a_2), a_3]

    def print_activations(self, features: np.ndarray) -> None:
        print(f'Printing activations for input: {features}')
        for index, array in enumerate(self.prop_forward(features)):
            print(f'Layer {index+1}: {array}')

    def get_deltas(self, y: np.ndarray, activations: List[np.ndarray]=None) -> List[np.ndarray]:
        # TO-DO: adapt code to accept different cost functions
        # Right now: hard coded to use quadratic loss
        if activations == None:
            activations = self.prop_forward(y)
        weights = self.get_weights(form='list')
        deltas = []
        deltas_output = -np.multiply((y-activations[-1]), np.apply_along_axis(sigmoid_derivative, 0, activations[-1]))
        deltas.insert(0, deltas_output)
        for i in range(len(activations)-2):
            delta = np.multiply(np.matmul(np.transpose(weights[-(i+2)]), deltas[-(i+1)]), np.apply_along_axis(sigmoid_derivative, 0, activations[-2]))
            deltas.insert(0, delta)
        return deltas

    def partial_derivatives(self, y: np.ndarray, derivative: Callable) -> List[np.ndarray]:
        activations = self.prop_forward(y)
        deltas = self.get_deltas(y, derivative, activations)
        partial_derivatives = []
        for index in range(len(self.layers)):
            next
            # To-Do:
        return

    def prop_backward(self, learning_rate: float=0.01) -> None:
        print('Performing backpropagation:')
        print(f'Old weights: {self.weights}')
        # TO-DO
        print(f'New weights: {self.weights}')
        return


test_network = Network([8,3,8], [True, True, False])

[array([-4.38708623e-06, -4.16987629e-06,  2.47459412e-06, -6.07341737e-06]), array([-0.11750197,  0.11750583,  0.11749433,  0.11749259,  0.11749311,
        0.11751673,  0.11749984,  0.11749342])]


In [71]:
test_network.print_weights()
#test_network.print_activations(X[0])
#print(test_network.get_deltas(y=X[0])

Printing weights of network:
Layer 1
[[-2.20416014e-05 -2.52834496e-05  1.49856583e-04 -1.15568236e-04
  -4.54890157e-06  1.82793447e-04 -1.53982959e-04  8.41859305e-05
   1.54359650e-05]
 [-3.47920980e-05  6.21081004e-05 -1.42811812e-05  4.37255526e-05
  -1.19643270e-04  1.98613284e-05  1.33909314e-04 -4.57509756e-05
   1.60639876e-04]
 [-2.26518058e-06  2.38637700e-05  1.31642177e-04 -1.18214784e-04
  -1.34834938e-04  7.71115766e-05  2.24699369e-05  8.69052901e-05
   7.02918260e-07]]
Layer 2
[[-1.76330414e-05  4.65598466e-06  5.69695344e-05 -2.97235847e-05]
 [ 1.29575556e-07 -2.43123818e-05  8.90422518e-05  8.91377601e-05]
 [ 8.00597873e-06 -1.69352609e-04 -6.95862093e-05 -6.91236761e-05]
 [-1.20472369e-04  9.54489746e-06 -4.41756895e-05 -8.39213582e-05]
 [-1.06473188e-04 -6.97281360e-05  1.07275408e-04 -1.63707986e-04]
 [ 2.34868690e-04 -1.73111288e-05  8.02324378e-05  4.45194563e-05]
 [-8.40050451e-05  7.73129844e-05  2.13757196e-05 -8.86488168e-06]
 [-1.39643333e-04  4.74859537e-0

In [123]:
def activation(inputs, weights, bias=True):
    
    if bias == True:
        term = 1
    else: 
        term = 0
    for input, weight in zip(inputs, weights):
        term += (input * weight)
    return sigmoid(term)



# activation function for top node hidden layer.
print(activation([1,0,0,0,0,0,0,0], np.zeros((8,1)), False))


[0.5]


In [121]:
def forward(network, input):
    for i in range(len(network)):
        if i == 0:
            network[i] = input
        elif i == 1:
            for node_i in range(len(network[i])):
                network[i][node_i] = activation(network[i-1], np.zeros(network[i-1].shape), bias=False)
        else: 
            for node_i in range(len(network[i])):
                network[i][node_i] = activation(network[i-1], np.zeros(network[i-1].shape))
    return network

In [184]:
sigmoid([1,0.5,0])

array([0.73105858, 0.62245933, 0.5       ])

In [194]:
activation = np.ones((1,8))
weights = np.ones((8,2))
np.dot(activation, weights)

array([[8., 8.]])